In [ ]:
import os
from pathlib import Path

notebook_dir = Path.cwd()
layout_dir = notebook_dir.parent.parent / "layout"

if Path.cwd().resolve() != layout_dir.resolve():
    os.chdir(layout_dir)
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
import pandas as pd
import warnings
from dowhen import when
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, str(notebook_dir))
from doubly_clamped_beam_comb_drive_tuning import doubly_clamped_beam_comb_drive_tuning as f

In [ ]:
def extract_wg(c:gf.Component):
    c_out = gf.Component()
    polys = c.get_polygons(layers=["WG"], merge=True)[gf.get_layer("WG")]
    if not polys:
        return c_out

    max_poly = max(polys, key=lambda p: p.area())
    c_out.add_polygon(max_poly, layer=gf.get_layer("WG"))
    return c_out

In [ ]:
import inspect

when(f, "beam_ref = c << beam").goto("shaft_ref = c.add_ref(shaft)")
when(f,"shaft_ref.connect").do("shaft_ref.move(origin=shaft_ref.center, destination=(0,0))").goto("spring_se = c << spring")
when(f,"beam_pad = U_shape_pad").goto('return c').do(inspect.getsource(extract_wg)).do("c= extract_wg(c)").do("""
c.info['shaft_width'] = shaft_width
""")

In [ ]:
params = {
    "shaft_height": 30,
    "beam_length": 1000,
    "finger_pair_num": 10,
    "combdrive_array_num_push": 1,
    "combdrive_array_num_pull": 6,
    "spring_length": 150,
    "spring_loop_num": 1,
    "combdrive_spacing": 70,
    "spring_width": 3,
    "finger_length": 15,
}
c = f(**params)

In [ ]:
c

In [ ]:
c.write_gds(notebook_dir / "shaft_sim.gds",with_metadata=False)

In [ ]:
import mph
client = mph.start()

In [8]:
model = client.load(notebook_dir / "shaft_beam_shell.mph")
jmodel = model.java

In [ ]:
model.parameter("shaft_w", f"{c.info['shaft_width']}[um]")
model.parameter("shaft_h", f"{params['shaft_height']}[um]")
model.parameter("num_pull", str(params['combdrive_array_num_pull']))
model.parameter("num_push", str(params['combdrive_array_num_push']))
model.parameter("spacing", f"{params['combdrive_spacing']}[um]")
model.parameter("xmax",f'{c.xmax}[um]')
model.parameter("xmin",f'-{c.xmax}[um]')
model.parameter("l_beam",f"{params['beam_length']}[um]")
model.parameter("total_force", "1.302e-05[N]")

In [ ]:
imp1 = model/'geometries'/'Geometry 1'/'Import 1'
imp1.java.importData()

In [ ]:
model.solve()

In [ ]:
model.save()

In [ ]:
mode_dir = notebook_dir / "mode_shape"
mode_dir.mkdir(exist_ok=True)
pg1 = model/'plots'/'Mode Shape (solid) 1'
img1 = model/'exports'/'Image 1'
import matplotlib.pyplot as plt

for i in range(1, 10):
    pg1.property("solnum", str(i))
    freq_val = model.evaluate("freq")[i - 1]
    fname = mode_dir / f"mode_{i}_freq_{freq_val:.3f}Hz.png"
    img1.property("filename", str(fname))
    model.export("Image 1", str(fname))

image_files = sorted(mode_dir.glob("mode_*_freq_*.png"))
n = len(image_files)
cols = 3
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 3 * rows))
axes = axes.ravel()

for ax, img_path in zip(axes, image_files):
    img = plt.imread(img_path)
    ax.imshow(img)
    ax.set_title(img_path.stem.replace("_", " "))
    ax.axis("off")

for ax in axes[n:]:
    ax.axis("off")

plt.tight_layout()
plt.show()

In [9]:
model.clear()
model.reset()
model.save()